# Asana Library

In [33]:
# | default_exp asana.user

In [34]:
# | exporti

from __future__ import annotations

from dataclasses import dataclass, field
from typing import List

from nbdev.showdoc import patch_to

import gdoc_sync.asana.auth as aa
import gdoc_sync.client as gd

In [45]:
# | export
@dataclass
class AsanaUser:
    id: str
    name: str
    auth: aa.AsanaAuth = field(repr=False)
    resource_type: str = field(repr=False)
    email: str = None

    @classmethod
    def _from_json(cls, obj, auth: aa.AsanaAuth):
        return cls(
            id=obj["gid"],
            name=obj["name"],
            email=obj.get("email"),
            resource_type=obj["resource_type"],
            auth=auth,
        )

In [46]:
@patch_to(AsanaUser, cls_method=True)
async def get_user(
    cls, auth: aa.AsanaAuth, debug_api: bool = False, return_raw: bool = False
) -> List[AsanaUser]:
    url = f"{auth.base_url}/users/"

    res = await gd.get_data(url, auth=auth, method="GET", debug_api=debug_api)

    if return_raw:
        return res

    return [cls._from_json(user_obj, auth=auth) for user_obj in res.response["data"]]

In [49]:
import os, json

# destructure auth fields from env
asana_pat = json.loads(os.environ["ASANA_PUBLIC"])["ASANA_PAT"]

workspace_id = "1206043185525448"

asana_auth = aa.AsanaAuth(token=asana_pat, workspace_id=workspace_id)

await AsanaUser.get_user(auth=asana_auth, debug_api=False, return_raw=False)

[AsanaUser(id='1206043147237379', name='Jae Wilson', email=None)]

In [50]:
# | exporti


@patch_to(AsanaUser, cls_method=True)
async def get_by_id(
    cls,
    user_id: str,
    auth: aa.AsanaAuth,
    debug_api: bool = False,
    return_raw: bool = False,
) -> AsanaUser:
    """Get a user by ID"""
    url = f"{auth.base_url}/users/{user_id}"

    res = await gd.get_data(url, auth=auth, method="GET", debug_api=debug_api)

    if return_raw:
        return res

    return cls._from_json(res.response["data"], auth=auth)

In [51]:
import os, json

# destructure auth fields from env
asana_pat = json.loads(os.environ["ASANA_PUBLIC"])["ASANA_PAT"]

workspace_id = "1206043185525448"
user_id = "1206043147237379"
asana_auth = aa.AsanaAuth(token=asana_pat, workspace_id=workspace_id)

await AsanaUser.get_by_id(user_id=user_id, auth=asana_auth, return_raw=False)

AsanaUser(id='1206043147237379', name='Jae Wilson', email='jaewilson07@gmail.com')

In [40]:
# | hide
import nbdev

nbdev.nbdev_export()